In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
# simulates the maximum of N standard exponential random variables
def expMax(N, lamb):
  rnd = -np.log(1 - np.random.uniform(0, 1) ** (1/N))
  return rnd

In [ ]:
#Testing simulation of gumbel variable

lamb = 0.5
N = 10**5
A, B, intervals = -2, 10, 60
width = (B - A)/intervals
freq = np.zeros(intervals)

for i in range(N):
  rnd = (-1/lamb) * math.log(np.random.uniform(0,1))
  index = (rnd + 2) * 5
  if (index >= 0 and index < intervals):
    freq[int(index)] += 1

freq = freq/(N * width)
#freq[0] = # of things between -2 and -1.99
xvalues = np.linspace(A, B, intervals)

yGumbel = lamb * np.exp(-1 * lamb * xvalues)


#plt.figure(figsize=(8,8))
plt.bar(xvalues, freq)
plt.plot(xvalues, yGumbel, color = "r")
plt.show()

In [ ]:
#Testing simulation of gumbel variable

N = 10**4
n = 10**2
lamb = 1
A, B, intervals = -2, 10, 60
width = (B - A)/intervals
freq = np.zeros(intervals)

for i in range(N):
  rnd = 0
  for j in range(n):
    rnd = max(rnd, (-1/lamb) * math.log(np.random.uniform(0,1)))
  rnd -= np.log(n)
  index = (rnd + 2) * 5
  if (index >= 0 and index < intervals):
    freq[int(index)] += 1

freq = freq/(N * width)
#freq[0] = # of things between -2 and -1.99
xvalues = np.linspace(A, B, intervals)
yGumbel = np.exp(-(xvalues + np.exp(-xvalues)))

#plt.figure(figsize=(8,8))
plt.bar(xvalues, freq)
plt.plot(xvalues, yGumbel, color = "r")
plt.show()

In [ ]:
#Testing simulation of max of D distributions

N = 10**5
n = 10**2
lamb = 1
A, B, intervals = -2, 10, 60
width = (B - A)/intervals
freq1 = np.zeros(intervals)
freq2 = np.zeros(intervals)


for i in range(N):
  rnd = -np.log(1 - np.random.uniform(0, 1) ** (1/n))
  index = (rnd + 2) * 5
  if (index >= 0 and index < intervals):
    freq1[int(index)] += 1

freq1 = freq1/(N * width)
#freq[0] = # of things between -2 and -1.99
xvalues = np.linspace(A, B, intervals)

for i in range(N):
  if (i % 10 == 0):
    print(i)
  rnd = expMax(n, lamb)
  index = (rnd + 2) * 5
  if (index >= 0 and index < intervals):
    freq2[int(index)] += 1

freq2 = freq2/(N * width)

freq3 = freq2 - freq1


plt.figure(figsize=(24,8))
plt.subplot(1, 3, 1)
plt.bar(xvalues, freq1)
plt.subplot(1, 3, 2)
plt.bar(xvalues, freq2)
plt.subplot(1, 3, 3)
plt.bar(xvalues, freq3)
plt.show()

In [ ]:
#simulates last passage percolation times, where the edges are the maximum of N standard exponential random variables

def simulate(N,n,m,lamb):
  T = np.zeros((n+1,m+1))
  U = np.zeros((n+1,m+1))
  V = np.zeros((n+1,m+1))
  for x in range(m+1):
    for y in range(n+1):
      U[x][y] = expMax(N,lamb)
      V[x][y] = expMax(N,lamb)

  T[1][1] = expMax2(N,lamb)
  for x in range(2,m+1):
    T[x][1] = T[x-1][1] + U[x][1]
  
  for y in range(2,n+1):
    T[1][y] = T[1][y-1] + V[1][y]
  
  for x in range(2,m+1):
    for y in range(2,n+1):
      T[x][y] = max(T[x-1][y] + U[x][y], T[x][y-1] + V[x][y])
  
  return T[n][m]

In [ ]:
#simulates last passage percolation times, where the edges are gumbel random variables

def calcLPT2(n,m):
  T = np.zeros((n+1,m+1))
  U = -np.log(-np.log(np.random.random((m + 1, n + 1))))
  V = -np.log(-np.log(np.random.random((m + 1, n + 1))))

  T[1][1] = -np.log(-np.log(np.random.uniform(0,1)))
  for x in range(2,m+1):
    T[x][1] = T[x-1][1] + U[x][1]
  
  for y in range(2,n+1):
    T[1][y] = T[1][y-1] + V[1][y]
  
  for x in range(2,m+1):
    for y in range(2,n+1):
      T[x][y] = max(T[x-1][y] + U[x][y], T[x][y-1] + V[x][y])
  
  return T[n][m]

In [ ]:
#numerically tests the results that the log of (x) * (m + n - 1), subtracted from the last passage percolation time (where edges are the maximum of N standard exponential random variables), approachse last passage percolation time of a gumbel random variable

def run(x):
  # m and n are coordinates
  m = 100
  n = 100
  lamb = 1

  nums = 100
  sum = 0
  for j in range(nums):
    sum += simulate(int(x), n, m, lamb) - np.log(x) * (m+n-1)
  return sum / nums

In [ ]:
#numerically tests the results that the log of (x) * (m + n - 1), subtracted from the last passage percolation time (where edges are the maximum of N standard exponential random variables), approachse last passage percolation time of a gumbel random variable
#uses the total variation between the two histograms to test

m = 100
n = 100
lamb = 1
x = 10000
nums = 10000
Yd = np.zeros(nums)
Yg = np.zeros(nums)
for i in range(nums):
  if (i % 100 == 0):
    print(i)
  Yd[i] = simulate(int(x), n, m, lamb) - np.log(x) * (m+n-1)
  Yg[i] = calcLPT2(m,n)

A, B = np.minimum(Yd, Yg), np.maximum(Yd, Yg)
hist1, _ = np.histogram(Yd, bins= int(np.sqrt(x)), density = True)
hist2, _ = np.histogram(Yg, bins= int(np.sqrt(x)), density = True)

totalVariationDistance = 0.5 * np.sum(np.absolute(hist1 - hist2))
print(totalVariationDistance)